In [1]:
# chat_sql_app.py

import streamlit as st
import psycopg2
from openai import OpenAI

# --- Page Config ---
st.set_page_config(page_title="Smart SQL & Chatbot", layout="centered")
st.title("💬 Ask Anything: AI + SQL Chatbot")

# --- Sidebar: Credentials ---
with st.sidebar:
    st.header("🔐 Configuration")
    openai_api_key = st.text_input("OpenAI API Key", type="password")
    db_host = st.text_input("Database Host", "host") # replace with your host
    db_user = st.text_input("Database User", "user") # replace with your user
    db_password = st.text_input("Database Password", type="password")
    db_name = st.text_input("Database Name", "postgres")
    db_port = st.number_input("Port", value=5432)

    if openai_api_key and db_host and db_user and db_password:
        try:
            if "client" not in st.session_state:
                st.session_state.client = OpenAI(api_key=openai_api_key)
                st.session_state.conn = psycopg2.connect(
                    host=db_host,
                    port=db_port,
                    dbname=db_name,
                    user=db_user,
                    password=db_password,
                    sslmode="require"
                )
                st.session_state.cur = st.session_state.conn.cursor()
            st.success("✅ Connected to Supabase")
        except Exception as e:
            st.error(f"❌ Connection failed: {e}")
    else:
        st.warning("🔑 Enter credentials to connect")


# --- Get table schema ---
def get_table_schema(table_name):
    cur = st.session_state.cur
    cur.execute("""
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = %s;
    """, (table_name,))
    return cur.fetchall()


# --- Ask GPT and Execute ---
def ask_any_question(question, table_name=None):
    client = st.session_state.client
    cur = st.session_state.cur

    table_info = ""
    table_hint = ""
    if table_name:
        schema = get_table_schema(table_name)
        if not schema:
            return f"⚠️ Table `{table_name}` not found.", []
        table_info = "\n".join([f"- {col} ({dtype})" for col, dtype in schema])
        table_hint = f"The table name is `{table_name}`. Use it exactly in the SQL query."

    system_prompt = f"""
You are a smart assistant that can either:
1. Generate SQL if the question is about data in a table.
2. Answer generally if it’s general knowledge.

Column names like years (e.g., 1943) must be in double quotes and included in the SELECT clause if used in ORDER BY.

When summing year columns like "1940" to "1945", assume they are numeric and write row-wise math like:
("1940" + "1941" + "1942" + "1943" + "1944" + "1945")

If a table is provided, only use SQL if the answer depends on its content.

Table schema:
{table_info}

{table_hint}

For the user question below, reply in one of two ways:
- "SQL:" followed by a SQL query using the correct table and column names
- "ANSWER:" followed by a natural language answer
"""

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ],
        temperature=0
    )
    reply = response.choices[0].message.content.strip()

    if reply.startswith("SQL:"):
        sql = reply[4:].strip()
        st.code(sql, language="sql")
        try:
            cur.execute(sql)
            rows = cur.fetchall()
            colnames = [desc[0] for desc in cur.description]
            return rows, colnames
        except Exception as e:
            return f"❌ SQL Error: {e}", []
    elif reply.startswith("ANSWER:"):
        return reply[7:].strip(), []
    else:
        return "🤖 Could not determine how to respond.", []


# --- Main App UI ---
if "client" in st.session_state:
    question = st.text_input("❓ Ask your question:")
    table_name = st.text_input("📋 Optional: Table name (if SQL related)")

    if st.button("Ask"):
        with st.spinner("Thinking..."):
            response, cols = ask_any_question(question, table_name if table_name else None)
            if isinstance(response, list):
                st.dataframe(response, use_container_width=True)
            else:
                st.write(response)


2025-05-30 20:15:25.015 
  command:

    streamlit run C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-05-30 20:15:25.020 Session state does not function when running a script without `streamlit run`
